In [1]:
import numpy as np
import scipy
import random
import tensorflow as tf
import time
import os
import gym
from gym import envs, scoreboard
from gym.spaces import Discrete, Box
import tempfile
import sys
from PGActorDiscrete import PGActorDiscrete

In [2]:

# ==========================
#   Training Parameters
# ==========================
# Max training steps
MAX_EPISODES = 10000
# Max episode length
MAX_EP_STEPS = 1000
# Base learning rate for the Actor network
ACTOR_LEARNING_RATE = 0.03
CRITIC_LEARNING_RATE = 0.1
# Discount factor 
GAMMA = 0.99

# ===========================
#   Utility Parameters
# ===========================
# Render gym env during training
RENDER_ENV = True
# Use Gym Monitor
GYM_MONITOR_EN = True
# Gym environment
ENV_NAME = 'Pendulum-v0'
# # Directory for storing gym results
# MONITOR_DIR = './results/gym_ddpg'
# # Directory for storing tensorboard summary results
# SUMMARY_DIR = './results/tf_ddpg'
RANDOM_SEED = 1337
MINIBATCH_SIZE = 200

In [ ]:
def train(sess, env, actor):
    
    # Initialize our Tensorflow variables
    sess.run(tf.initialize_all_variables())

    # Initialize memory
    old_states = []
    old_actions = []
    old_advantages = []
    old_hindsight_values = []
    mem_fill = 0
    nep_reward = 0
    nep_est_reward = 0
    
    for i in range(MAX_EPISODES):
        
        s = env.reset()
        v = actor.predict_value(np.reshape(s, (1, -1)))[0]
        nep_est_reward += v
        ep_reward = 0
        advantages = []
        
        for j in range(MAX_EP_STEPS):
            
            if RENDER_ENV:
                env.render()
            
            # get the probabilities and sample for an action
            probs = actor.predict(np.reshape(s, (1, -1)))
            a = [np.random.choice(range(actor.a_dim+1), p=probs[0])]
                                  
            # get new state and reward
            s2, r, is_done, info = env.step(a[0])
            
            # get new value prediction and find delta
            v2 = actor.predict_value(np.reshape(s2, (1, -1)))[0]
            
            if is_done:
                hs_v = [r]
            else:
                hs_v = r + GAMMA * v2
                
            # add step to batch
            old_states.append(np.reshape(s, (actor.s_dim,)))
            old_actions.append(np.reshape(a, (actor.a_dim,)))
            old_hindsight_values.append(hs_v)
            advantages.append(-v) # this will get modified in future timesteps
            for k in range(len(advantages)):
                assert k <= j
                advantages[k] += r * (GAMMA**(j-k))
            
            s=s2
            v=v2
            ep_reward += r
            
            # end of episode methods
            if is_done or j == MAX_EP_STEPS-1: 
                ep_len = j+1
                mem_fill += ep_len
                old_advantages = old_advantages + advantages
                nep_reward += ep_reward
                break
                                  
        if i % 100 ==0 and i!=0:
                print '| Avg value (100 eps): %.2i' % (int(nep_reward/100)), \
                " | Avg est value (100 eps): ", nep_est_reward[0]/100, " | Episode", i
                print
                nep_reward=0
                nep_est_reward = 0
        if mem_fill >= MINIBATCH_SIZE:
            
#             print "Training on", mem_fill, "examples"
#             print np.asarray(old_hindsight_values).shape
            actor.train(old_states, old_actions, old_advantages)
            actor.train_value(old_states, old_hindsight_values)
            old_states = []
            old_actions = []
            old_advantages = []
            old_hindsight_values = []
                                
            mem_fill = 0
            

In [ ]:
# defining environment
sess = tf.Session()

env = gym.make('CartPole-v0')
# print env.observation_space
# state_dim = env.observation_space.shape[0]
# action_dim = env.action_space
# print action_dim
# print "state and actions dims:", state_dim, action_dim

# make sure action bound is symmetric (can change in future,
# but need to remember to scale actor output appropriately)
# assert (env.action_space.high == -env.action_space.low)

# action_bound = env.action_space.high

# start up actor and critic pair

actor = PGActorDiscrete(sess, 4, 1, 
                 ACTOR_LEARNING_RATE, CRITIC_LEARNING_RATE)

train(sess, env, actor)

[2016-11-22 18:42:17,021] Making new env: CartPole-v0


| Avg value (100 eps): 21  | Avg est value (100 eps):  11.7508044434  | Episode 100

| Avg value (100 eps): 38  | Avg est value (100 eps):  25.2199462891  | Episode 200

| Avg value (100 eps): 133  | Avg est value (100 eps):  50.9493505859  | Episode 300

| Avg value (100 eps): 133  | Avg est value (100 eps):  55.2392285156  | Episode 400

| Avg value (100 eps): 182  | Avg est value (100 eps):  77.3533935547  | Episode 500

